In [3]:
pip install geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 2.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [44]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from geopy.distance import geodesic


In [75]:
# Load data
ward_data = pd.read_csv('/Users/jay/Desktop/DISS/DISS_Data/wards_data.csv')
fire_stations = pd.read_csv('/Users/jay/Desktop/DISS/DISS_Data/stations.csv')
fire_stations['station_id'] = fire_stations.index

In [76]:
# Define demand locations and potential fire station sites
demand_locations = ward_data['ward_id'].tolist()
potential_fire_station_sites = fire_stations['station_id'].tolist()

# Define demand weights
demand_weights = ward_data.set_index('ward_id')['population'].to_dict()

In [77]:
# Extract coordinates
fs_coordinates = fire_stations[['latitude', 'longitude']].values
ward_coordinates = ward_data['centroid'].apply(lambda x: tuple(map(float, x.strip('POINT ()').split()[::-1]))).values


# Calculate distance matrix
distance_matrix = np.zeros((len(ward_coordinates), len(fs_coordinates)))

for i, wc in enumerate(ward_coordinates):
    for j, fc in enumerate(fs_coordinates):
        distance_matrix[i, j] = geodesic(wc, fc).kilometers


In [78]:
# Create a new model
model = gp.Model('MCLP')

# Decision variables
x = model.addVars(demand_locations, vtype=GRB.BINARY, name='X')  # Whether a ward is covered
y = model.addVars(potential_fire_station_sites, vtype=GRB.BINARY, name='Y')  # Whether a station is sited


In [79]:
# Objective: Maximize the total weighted demand covered by selected fire stations
objective = gp.quicksum(demand_weights[ward] * x[ward] for ward in demand_locations)
model.setObjective(objective, GRB.MAXIMIZE)


In [87]:
# Constraints
coverage_distance = 2.5 # km

# Coverage constraint: Ensure that if a demand location is covered, at least one fire station within the coverage distance must be sited
for i, ward in enumerate(demand_locations):
    covered_stations = [j for j in potential_fire_station_sites if distance_matrix[i, j] <= coverage_distance]
    print(f"Ward {ward} can be covered by stations: {covered_stations}")  # Debug information
    model.addConstr(gp.quicksum(y[j] for j in covered_stations) >= x[ward], name=f"coverage_{ward}")


Ward E05000405 can be covered by stations: []
Ward E05000414 can be covered by stations: [63]
Ward E05000401 can be covered by stations: [62, 63]
Ward E05000400 can be covered by stations: [63]
Ward E05000402 can be covered by stations: [64]
Ward E05000406 can be covered by stations: []
Ward E05000404 can be covered by stations: []
Ward E05000413 can be covered by stations: [63]
Ward E05000410 can be covered by stations: [64]
Ward E05000412 can be covered by stations: [62, 63]
Ward E05000408 can be covered by stations: [62, 63]
Ward E05000403 can be covered by stations: [62]
Ward E05000409 can be covered by stations: [62, 63]
Ward E05000407 can be covered by stations: [62, 64]
Ward E05000411 can be covered by stations: [64]
Ward E05000415 can be covered by stations: [62]
Ward E05011466 can be covered by stations: [23]
Ward E05011476 can be covered by stations: [23]
Ward E05011487 can be covered by stations: [21, 88]
Ward E05011474 can be covered by stations: []
Ward E05011469 can be co

In [88]:
# Number of fire stations constraint: Specify the number of fire stations to be established
number_of_fire_stations = 102  # Adjust based on your requirement
model.addConstr(gp.quicksum(y[station] for station in potential_fire_station_sites) == number_of_fire_stations, "num_stations")


<gurobi.Constr *Awaiting Model Update*>

In [89]:

# Optimize the model
model.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.5.0 22F82)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1268 rows, 745 columns and 4449 nonzeros
Model fingerprint: 0xabce8264
Variable types: 0 continuous, 745 integer (745 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+03, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint coverage_E05000406 by 1.000000000

Found heuristic solution: objective 7547842.0000
Presolve removed 1033 rows and 415 columns
Presolve time: 0.11s
Presolved: 235 rows, 330 columns, 1144 nonzeros
Variable types: 0 continuous, 330 integer (329 binary)
Found heuristic solution: objective 7741342.0000

Root relaxation: objective 7.925418e+06, 48 iterations,

In [90]:
# Print the results
if model.status == GRB.OPTIMAL:
    selected_stations = [station for station in potential_fire_station_sites if y[station].x > 0.5]
    print("Selected Fire Stations:", selected_stations)
    
    covered_wards = [ward for ward in demand_locations if x[ward].x > 0.5]
    print("Covered Wards:", covered_wards)
    
    # Additional analysis (optional)
    covered_demand = sum(demand_weights[ward] for ward in covered_wards)
    print(f"Total covered demand: {covered_demand}")
    # Print out the full details of each selected station and the wards they cover
    for station in selected_stations:
        covered_by_station = [ward for i, ward in enumerate(demand_locations) if distance_matrix[i, station] <= coverage_distance]
        print(f"Fire Station {station} covers wards: {covered_by_station}")

else:
    print("No optimal solution found.")

Selected Fire Stations: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 61, 62, 63, 64, 65, 66, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 108, 109, 110]
Covered Wards: ['E05000414', 'E05000401', 'E05000400', 'E05000402', 'E05000413', 'E05000410', 'E05000412', 'E05000408', 'E05000403', 'E05000409', 'E05000407', 'E05000411', 'E05000415', 'E05011466', 'E05011476', 'E05011487', 'E05011469', 'E05011477', 'E05011481', 'E05011484', 'E05011480', 'E05011485', 'E05011483', 'E05011482', 'E05011465', 'E05011488', 'E05011464', 'E05011472', 'E05011473', 'E05011471', 'E05011468', 'E05011475', 'E05011463', 'E05011462', 'E05011470', 'E05011479', 'E05011489', 'E05011486', 'E05011467', 'E05000117', 'E05000110', 'E05000107', 'E050

In [98]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from geopy.distance import geodesic

# Load data
ward_data = pd.read_csv('/Users/jay/Desktop/DISS/DISS_Data/wards_data.csv')
fire_stations = pd.read_csv('/Users/jay/Desktop/DISS/DISS_Data/stations.csv')
fire_stations['station_id'] = fire_stations.index

# Define demand locations and potential fire station sites
demand_locations = ward_data['ward_id'].tolist()
potential_fire_station_sites = fire_stations['station_id'].tolist()

# Define demand weights
demand_weights = ward_data.set_index('ward_id')['demand_weight'].to_dict()

# Extract coordinates
fs_coordinates = fire_stations[['latitude', 'longitude']].values
ward_coordinates = ward_data['centroid'].apply(lambda x: tuple(map(float, x.strip('POINT ()').split()[::-1]))).values


# Calculate distance matrix
distance_matrix = np.zeros((len(ward_coordinates), len(fs_coordinates)))

for i, wc in enumerate(ward_coordinates):
    for j, fc in enumerate(fs_coordinates):
        distance_matrix[i, j] = geodesic(wc, fc).kilometers


# Create a new model
model = gp.Model('MCLP')

# Decision variables
x = model.addVars(demand_locations, vtype=GRB.BINARY, name='X')  # Whether a ward is covered
y = model.addVars(potential_fire_station_sites, vtype=GRB.BINARY, name='Y')  # Whether a station is sited

# Objective: Maximize the total weighted demand covered by selected fire stations
objective = gp.quicksum(demand_weights[ward] * x[ward] for ward in demand_locations)
model.setObjective(objective, GRB.MAXIMIZE)

# Constraints
coverage_distance = 2.5 # km

# Coverage constraint: Ensure that if a demand location is covered, at least one fire station within the coverage distance must be sited
for i, ward in enumerate(demand_locations):
    covered_stations = [j for j in potential_fire_station_sites if distance_matrix[i, j] <= coverage_distance]
    
# Number of fire stations constraint: Specify the number of fire stations to be established
number_of_fire_stations = 102  # Adjust based on your requirement
model.addConstr(gp.quicksum(y[station] for station in potential_fire_station_sites) == number_of_fire_stations, "num_stations")

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    selected_stations = [station for station in potential_fire_station_sites if y[station].x > 0.5]
    print("Selected Fire Stations:", selected_stations)
    
    covered_wards = [ward for ward in demand_locations if x[ward].x > 0.5]
    print("Covered Wards:", covered_wards)
    
    # Print out the full details of each selected station and the wards they cover
    for station in selected_stations:
        covered_by_station = [ward for i, ward in enumerate(demand_locations) if distance_matrix[i, station] <= coverage_distance]
        print(f"Fire Station {station} covers wards: {covered_by_station}")

else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.5.0 22F82)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 745 columns and 112 nonzeros
Model fingerprint: 0xd10a1393
Variable types: 0 continuous, 745 integer (745 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective 5218446.8000

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 5.21845e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.218446800000e+06, best bound 5.218446800000e+06, gap 0.0000%
Selected Fire Stations: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3

Fire Station 61 covers wards: ['E05000092', 'E05000098', 'E05000087', 'E05000094', 'E05000265', 'E05000263', 'E05000253', 'E05009397', 'E05009398', 'E05009399', 'E05009404', 'E05009392', 'E05009394', 'E05009396', 'E05009390', 'E05009400', 'E05000638', 'E05000635', 'E05000642', 'E05000631', 'E05000648', 'E05000639', 'E05000640']
Fire Station 62 covers wards: ['E05000401', 'E05000412', 'E05000408', 'E05000403', 'E05000409', 'E05000407', 'E05000415', 'E05000530', 'E05000522']
Fire Station 63 covers wards: ['E05000414', 'E05000401', 'E05000400', 'E05000413', 'E05000412', 'E05000408', 'E05000409']
Fire Station 64 covers wards: ['E05000402', 'E05000410', 'E05000407', 'E05000411', 'E05000572', 'E05000456', 'E05000465', 'E05000473', 'E05000469']
Fire Station 65 covers wards: ['E05000418', 'E05000417', 'E05000435', 'E05000420', 'E05000421', 'E05000425', 'E05000426', 'E05000436', 'E05000423', 'E05000419', 'E05000429', 'E05011100', 'E05011097', 'E05011096', 'E05011115', 'E05011105', 'E05011103']


In [95]:
selected_stations = [station for station in potential_fire_station_sites if y[station].x > 0.5]
print("Selected Fire Stations:", selected_stations)
    
covered_wards = [ward for ward in demand_locations if x[ward].x > 0.5]
print("Covered Wards:", covered_wards)

Selected Fire Stations: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]
Covered Wards: ['E05000405', 'E05000414', 'E05000401', 'E05000400', 'E05000402', 'E05000406', 'E05000404', 'E05000413', 'E05000410', 'E05000412', 'E05000408', 'E05000403', 'E05000409', 'E05000407', 'E05000411', 'E05000415', 'E05011466', 'E05011476', 'E05011487', 'E05011474', 'E05011469', 'E05011477', 'E05011478', 'E05011481', 'E05011484', 'E05011480', 'E05011485', 'E05011483', 'E05011482', 'E05011465', 'E05011488', 'E05011464', 'E05011472', 'E05011473', 'E05011471', 'E05011468', 'E05011475', 'E05011463', 'E05011462', 'E05011470', 'E05011479', 'E05011489', 'E05011486', 

In [97]:

# Extract the deleted fire stations (not in selected_stations)
all_station_ids = fire_stations.index.tolist()
selected_stations = [station for station in all_station_ids if station in selected_stations]

# Filter the datasets based on the results
selected_stations_df = fire_stations.loc[selected_stations]
covered_wards_df = ward_data[ward_data['ward_id'].isin(covered_wards)]

# Save the filtered datasets
selected_stations_df.to_csv('selected_stations1.csv', index=False)
covered_wards_df.to_csv('covered_wards1.csv', index=False)


selected_stations_df.head(), covered_wards_df.head()

(        name                                            address  \
 0   Dagenham  Rainham Road North, Dagenham, Greater London R...   
 1    Barking                   Alfred's Way, Greater London, UK   
 2     Barnet   144 Station Road, Barnet, Greater London EN5, UK   
 3  Mill Hill  10 Hartley Avenue, Edgware, London, Greater Lo...   
 4   Finchley  227 Long Lane, London Borough of Barnet, Londo...   
 
                 borough   latitude  longitude  station_id  
 0  Barking and Dagenham  51.558785   0.157054           0  
 1  Barking and Dagenham  51.532104   0.103288           1  
 2                Barnet  51.646995  -0.185843           2  
 3                Barnet  51.615091  -0.243472           3  
 4                Barnet  51.597836  -0.179377           4  ,
                      NAME    ward_id              DISTRICT  LAGSSCODE  \
 0       Chessington South  E05000405  Kingston upon Thames  E09000021   
 1  Tolworth and Hook Rise  E05000414  Kingston upon Thames  E09000021   
 